# Analyzing DeFi Data on Ethereum Blockchain to Understand Behaviors
## Preprocessing

In [ ]:
# On Kaggle
import pandas as pd

def load_data(list, base_path='/kaggle/input/defi-protocol-data-on-ethereum-2yr-23-to-24/dataset/data'):
    dataframes = {}
    for file in list:
        file_path = f"{base_path}/{file}.parquet"
        df = pd.read_parquet(file_path)
        dataframes[file] = df
        print(f"DataFrame '{file}' contains columns:\n{df.columns.tolist()}\n")
    return dataframes

dataframes = load_data(['users', 'market'])

for key, df in dataframes.items():
    print(f"Dataframe {key}:\n=============================================\n")
    print(f"{df.head(5)}\n")

In [1]:
# On Local
import pandas as pd

def load_data(list, base_path='../data/dataset'):
    dataframes = {}
    for file in list:
        file_path = f"{base_path}/{file}.parquet"
        df = pd.read_parquet(file_path)
        dataframes[file] = df
        print(f"DataFrame '{file}' contains columns:\n{df.columns.tolist()}\n")
    return dataframes

dataframes = load_data(['users', 'market'])

for key, df in dataframes.items():
    print(f"Dataframe {key}:\n=============================================\n")
    print(f"{df.head(5)}\n")

DataFrame 'users' contains columns:
['address', 'first_seen', 'last_seen', 'protocol_types', 'protocols_used', 'received_count', 'total_received (ETH)', 'sent_count', 'total_sent (ETH)', 'transactions']

DataFrame 'market' contains columns:
['timestamp', 'blockchain', 'protocol_name', 'symbol', 'type', 'contract_address', 'open (usd)', 'high (usd)', 'low (usd)', 'close (usd)', 'volume', 'nb_tx_1h', 'nb_tx_24h', 'total_value_eth_1h', 'total_value_eth_24h', 'total_gas_used_1h', 'total_gas_used_24h', 'nb_unique_receivers_1h', 'nb_unique_receivers_24h', 'nb_unique_senders_1h', 'nb_unique_senders_24h', 'std_value_eth_1h', 'std_value_eth_24h', 'std_gas_used_1h', 'std_gas_used_24h', 'avg_gas_used_1h', 'avg_gas_used_24h', 'avg_value_eth_per_tx_1h', 'avg_value_eth_per_tx_24h', 'max_gas_used_1h', 'max_gas_used_24h', 'max_value_eth_1h', 'max_value_eth_24h', 'median_value_eth_1h', 'median_value_eth_24h', 'min_gas_used_1h', 'min_gas_used_24h', 'min_value_eth_1h', 'min_value_eth_24h', 'num_errors_1h

In [2]:
def clean_column_names(df):
    """
    Clean column names by stripping leading/trailing spaces, lowercasing all characters, and replacing spaces with underscores.
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

## 1) Extraction of protocol types/names and count

In [3]:
import json

users = dataframes['users'].head(10000).copy()
market = dataframes['market'].copy()

#---------------------------------------------- Protocol Types

def parse_protocols(protocol_column):
    """
    Parses a JSON string into a dictionary. If the string is not a valid JSON, returns an empty dictionary.
    """
    try:
        protocols = json.loads(protocol_column)
        return protocols if isinstance(protocols, dict) else {}
    except json.JSONDecodeError:
        return {}

def process_user_protocols(users):
    """
    Processes user protocols by extracting specific protocol types and adding them as separate columns.
    """
    protocol_columns = ['type_' + k for k in ['DEX', 'Lending', 'Stablecoin', "Yield Farming", "NFT-Fi"]]
    users = users.assign(**{col: 0 for col in protocol_columns})

    users['parsed_protocols'] = users['protocol_types'].apply(parse_protocols)

    for protocol in ['DEX', 'Lending', 'Stablecoin', "Yield Farming", "NFT-Fi"]:
        column_name = f'type_{protocol}'
        users[column_name] = users['parsed_protocols'].apply(lambda x: x.get(protocol, 0))
    
    return users.drop(columns=['protocol_types', 'parsed_protocols'])

users = process_user_protocols(users)

#---------------------------------------------- Protocl Used

def transform_protocols_column(df, column_name='protocols_used'):
    """
    Transforms the 'protocols_used' column by converting string representations of dictionaries into actual dictionaries.
    Then, it creates new columns for each protocol used, counting the occurrences of each protocol.
    """
    
    df[column_name] = df[column_name].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    count_columns = []
    
    for index, row in df.iterrows():
        protocols = row[column_name]
        for protocol_name, protocol_data in protocols.items():
            count_columns.append(f'{protocol_name}_count')
            df.at[index, f'{protocol_name}_count'] = int(protocol_data.get('count', 0))
    
    return df.drop(columns=[column_name])

users = transform_protocols_column(users)

#---------------------------------------------- Cleaning

users = clean_column_names(users)
users.fillna(0, inplace=True)
protocols_counts = ['curve_dao_count', 'aave_count', 'tether_count', 'uniswap_count', 'maker_count', 'yearn.finance_count', 'usdc_count', 'dai_count', 'balancer_count', 'harvest_finance_count']
users[protocols_counts] = users[protocols_counts].astype(int)

print(users.columns)
users.head(10)

Index(['address', 'first_seen', 'last_seen', 'received_count',
       'total_received_(eth)', 'sent_count', 'total_sent_(eth)',
       'transactions', 'type_dex', 'type_lending', 'type_stablecoin',
       'type_yield_farming', 'type_nft-fi', 'curve_dao_count', 'aave_count',
       'tether_count', 'uniswap_count', 'maker_count', 'yearn.finance_count',
       'usdc_count', 'dai_count', 'balancer_count', 'harvest_finance_count'],
      dtype='object')


,address,first_seen,last_seen,received_count,total_received_(eth),sent_count,total_sent_(eth),transactions,type_dex,type_lending,...,curve_dao_count,aave_count,tether_count,uniswap_count,maker_count,yearn.finance_count,usdc_count,dai_count,balancer_count,harvest_finance_count
0,0x2deacac3a34ace696d3fe214a07882949311e81b,2023-01-07 22:05:35,2023-01-21 12:46:35,2,2.300373e+01,0,0.000000e+00,"[{""transaction_hash"": ""0x7de2510bd9efdb0fcbb35...",1,1,...,1,1,0,0,0,0,0,0,0,0
1,0x5e57eb8d7526f0b21922a3dad5abed627a0eaa33,2024-05-30 15:42:23,2024-05-30 16:25:11,0,0.000000e+00,2,6.030090e-10,"[{""transaction_hash"": ""0x4c07a83613579266cb163...",0,0,...,0,0,2,0,0,0,0,0,0,0
2,0x28cf5d293dc8eb3f1c35584171db1119aebce290,2024-01-11 08:41:59,2024-01-11 08:41:59,1,2.740758e+01,0,0.000000e+00,"[{""transaction_hash"": ""0xc71565381ba2b01d0657e...",1,0,...,0,0,0,1,0,0,0,0,0,0
3,0xb78429cbaf800905aac536fbe0a1c56c9e011e29,2024-01-11 08:42:11,2024-01-11 08:43:35,1,1.483507e+02,1,1.483507e+02,"[{""transaction_hash"": ""0x3b191d8163006073acec8...",2,0,...,0,0,0,2,0,0,0,0,0,0
4,0xa2055dff035a2de0825003e6c4b37af1d7149722,2024-01-04 21:10:35,2024-01-04 21:10:35,1,7.984539e-03,0,0.000000e+00,"[{""transaction_hash"": ""0x1ff61cf9322e10d01b18a...",0,1,...,0,0,0,0,1,0,0,0,0,0
5,0xca8a4c0c3409106f6727a40ebe2b664b5e9a0a40,2023-03-22 13:53:47,2023-03-23 15:29:47,2,2.476000e+02,0,0.000000e+00,"[{""transaction_hash"": ""0x7a214215575369582e4ac...",2,0,...,1,0,0,1,0,0,0,0,0,0
6,0x6a2b2418269af7a76fb88e6591496ca46162b751,2024-03-08 07:48:47,2024-03-09 17:15:35,2,2.630438e-08,1,2.042355e-08,"[{""transaction_hash"": ""0x5d8e2846e0474c416d6e3...",0,0,...,0,0,3,0,0,0,0,0,0,0
7,0xd7a57596ea1626dfb872423c7a4f7e343314bd9a,2024-01-11 08:46:35,2024-12-07 04:48:11,1,1.470585e+03,9,1.470585e+03,"[{""transaction_hash"": ""0x07ccc2c1f6ba2b8312780...",10,0,...,0,0,0,10,0,0,0,0,0,0
8,0xc57eb178182c06ec16d6b54821cc0708675bb365,2024-01-11 08:46:47,2024-01-22 16:33:23,2,6.682300e+02,1,6.682300e+02,"[{""transaction_hash"": ""0x98beacebfe87a81235eff...",3,0,...,0,0,0,3,0,0,0,0,0,0
9,0xba7c6441acfdbf46ef2cf6864a3549c471d252ac,2023-11-12 02:44:23,2023-11-12 02:44:23,1,1.251530e-09,0,0.000000e+00,"[{""transaction_hash"": ""0x276e8d4de4ca9b75dffb1...",0,0,...,0,0,1,0,0,0,0,0,0,0


## 2) Extract transactions data

In [4]:
def extract_transactions(row):
    transactions = json.loads(row['transactions'])  # Convertir la chaîne JSON en liste de dictionnaires
    extracted_rows = []
    
    for txn in transactions:
        extracted_row = {
            'address': row['address'],
            'first_seen': row['first_seen'],
            'last_seen': row['last_seen'],
            'received_count': row['received_count'],
            'total_received_(eth)': row['total_received_(eth)'],
            'sent_count': row['sent_count'],
            'total_sent_(eth)': row['total_sent_(eth)'],
            'transactions': row['transactions'],  
            'type_dex': row['type_dex'],
            'type_lending': row['type_lending'],
            'type_stablecoin': row['type_stablecoin'],
            'type_yield_farming': row['type_yield_farming'],
            'type_nft-fi': row['type_nft-fi'],
            'curve_dao_count': row['curve_dao_count'],
            'aave_count': row['aave_count'],
            'tether_count': row['tether_count'],
            'uniswap_count': row['uniswap_count'],
            'maker_count': row['maker_count'],
            'yearn.finance_count': row['yearn.finance_count'],
            'usdc_count': row['usdc_count'],
            'dai_count': row['dai_count'],
            'balancer_count': row['balancer_count'],
            'harvest_finance_count': row['harvest_finance_count'],
            'tx_timestamp': txn['timestamp'],
            'tx_protocol': txn['protocol_name'],
            'tx_value_(eth)': txn['value (ETH)'],
            'tx_is_sender': txn['is_sender'],
            'tx_gas_used': txn['gas_used']
        }
        extracted_rows.append(extracted_row)
    return extracted_rows

expanded_rows = users.apply(extract_transactions, axis=1)

users = pd.DataFrame([item for sublist in expanded_rows for item in sublist])

users.drop(columns=['transactions'], inplace=True)
print(users.columns)
users.head(5)

Index(['address', 'first_seen', 'last_seen', 'received_count',
       'total_received_(eth)', 'sent_count', 'total_sent_(eth)', 'type_dex',
       'type_lending', 'type_stablecoin', 'type_yield_farming', 'type_nft-fi',
       'curve_dao_count', 'aave_count', 'tether_count', 'uniswap_count',
       'maker_count', 'yearn.finance_count', 'usdc_count', 'dai_count',
       'balancer_count', 'harvest_finance_count', 'tx_timestamp',
       'tx_protocol', 'tx_value_(eth)', 'tx_is_sender', 'tx_gas_used'],
      dtype='object')


,address,first_seen,last_seen,received_count,total_received_(eth),sent_count,total_sent_(eth),type_dex,type_lending,type_stablecoin,...,yearn.finance_count,usdc_count,dai_count,balancer_count,harvest_finance_count,tx_timestamp,tx_protocol,tx_value_(eth),tx_is_sender,tx_gas_used
0,0x2deacac3a34ace696d3fe214a07882949311e81b,2023-01-07 22:05:35,2023-01-21 12:46:35,2,23.003725,0,0.000000e+00,1,1,0,...,0,0,0,0,0,2023-01-07 22:05:35,Curve DAO,2.300000e+01,False,34225.0
1,0x2deacac3a34ace696d3fe214a07882949311e81b,2023-01-07 22:05:35,2023-01-21 12:46:35,2,23.003725,0,0.000000e+00,1,1,0,...,0,0,0,0,0,2023-01-21 12:46:35,Aave,3.725116e-03,False,237828.0
2,0x5e57eb8d7526f0b21922a3dad5abed627a0eaa33,2024-05-30 15:42:23,2024-05-30 16:25:11,0,0.000000,2,6.030090e-10,0,0,2,...,0,0,0,0,0,2024-05-30 15:42:23,Tether,5.030090e-10,True,46109.0
3,0x5e57eb8d7526f0b21922a3dad5abed627a0eaa33,2024-05-30 15:42:23,2024-05-30 16:25:11,0,0.000000,2,6.030090e-10,0,0,2,...,0,0,0,0,0,2024-05-30 16:25:11,Tether,1.000000e-10,True,267393.0
4,0x28cf5d293dc8eb3f1c35584171db1119aebce290,2024-01-11 08:41:59,2024-01-11 08:41:59,1,27.407580,0,0.000000e+00,1,0,0,...,0,0,0,0,0,2024-01-11 08:41:59,Uniswap,2.740758e+01,False,57218.0


In [5]:
print(market.columns)

Index(['timestamp', 'blockchain', 'protocol_name', 'symbol', 'type',
       'contract_address', 'open (usd)', 'high (usd)', 'low (usd)',
       'close (usd)', 'volume', 'nb_tx_1h', 'nb_tx_24h', 'total_value_eth_1h',
       'total_value_eth_24h', 'total_gas_used_1h', 'total_gas_used_24h',
       'nb_unique_receivers_1h', 'nb_unique_receivers_24h',
       'nb_unique_senders_1h', 'nb_unique_senders_24h', 'std_value_eth_1h',
       'std_value_eth_24h', 'std_gas_used_1h', 'std_gas_used_24h',
       'avg_gas_used_1h', 'avg_gas_used_24h', 'avg_value_eth_per_tx_1h',
       'avg_value_eth_per_tx_24h', 'max_gas_used_1h', 'max_gas_used_24h',
       'max_value_eth_1h', 'max_value_eth_24h', 'median_value_eth_1h',
       'median_value_eth_24h', 'min_gas_used_1h', 'min_gas_used_24h',
       'min_value_eth_1h', 'min_value_eth_24h', 'num_errors_1h',
       'num_errors_24h', 'error_rate_1h', 'error_rate_24h'],
      dtype='object')


In [6]:
print(users['tx_protocol'].unique())
print(market['protocol_name'].unique())

['Curve DAO' 'Aave' 'Tether' 'Uniswap' 'Maker' 'yearn.finance' 'USDC'
 'Dai' 'Balancer' 'Harvest Finance']
['NFTFI' 'Harvest Finance' 'Balancer' 'yearn.finance' 'Maker' 'Curve DAO'
 'Aave' 'Uniswap' 'Dai' 'USDC' 'Tether']


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp

def merge_users_with_market(df_users, df_market, batch_size=1000):
    df_users['tx_timestamp'] = pd.to_datetime(df_users['tx_timestamp'])
    df_market['timestamp'] = pd.to_datetime(df_market['timestamp'])

    market_columns = [col for col in df_market.columns if col not in ['timestamp', 'protocol_name']]

    merged_df = df_users.copy()
    for col in market_columns:
        merged_df[col] = str(np.nan)
    
    grouped_market = df_market.groupby('protocol_name')

    for batch_start in tqdm(range(0, len(df_users), batch_size), desc="Processing users", unit="lot"):
        batch_end = min(batch_start + batch_size, len(df_users))
        batch = df_users.iloc[batch_start:batch_end]
        
        for idx, user_row in batch.iterrows():
            protocol = user_row['tx_protocol']
            tx_timestamp = user_row['tx_timestamp']

            if protocol in grouped_market.groups:
                relevant_market_data = grouped_market.get_group(protocol).copy()

                relevant_market_data['time_diff'] = abs(relevant_market_data['timestamp'] - tx_timestamp)
                closest_market_row = relevant_market_data.loc[relevant_market_data['time_diff'].idxmin()]

                for col in market_columns:
                    merged_df.at[idx, col] = closest_market_row[col]
    
    return merged_df

data = merge_users_with_market(users, market)
print(data.columns)
print(data.head(5))

Processing users:   8%|▊         | 25/331 [00:47<09:57,  1.95s/lot]

In [ ]:
data.to_csv('data.csv', index=False)